In [1]:
!pip install ast

ERROR: Could not find a version that satisfies the requirement ast (from versions: 0.0.1, 0.0.2)
ERROR: No matching distribution found for ast


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import ast
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



---

# Importing the datasets

---



In [4]:
df_links_small = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/links_small.csv')

In [5]:
df_links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
df_keywords = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/keywords.csv')

In [7]:
df_keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [8]:
df_credits = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/credits.csv')

In [9]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [10]:
df_movies_metadata_fixed = pd.read_csv('/content/drive/MyDrive/DataSets/the-movies-dataset/movies_metadata_fixed.csv')

In [11]:
df_movies_metadata_fixed.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415




---
# Feature selection
---





In [12]:
df_movies_metadata_fixed = df_movies_metadata_fixed[['id', 'title', 'release_date', 'vote_count', 'vote_average', 'genres']]
df_movies_metadata_fixed.head()

,id,title,release_date,vote_count,vote_average,genres
0,862,Toy Story,1995-10-30,5415,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,1995-12-15,2413,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,1995-12-22,92,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,1995-12-22,34,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,1995-02-10,173,5.7,"[{'id': 35, 'name': 'Comedy'}]"


In [13]:
df = df_movies_metadata_fixed.merge(df_credits, how='inner', on='id')
df = df.merge(df_keywords, how='inner', on='id')

In [14]:
df_links_small = df_links_small.rename(columns={'tmdbId': 'id'})

In [15]:
df = df.merge(df_links_small, how='inner', on='id')
df.head(1)

,id,title,release_date,vote_count,vote_average,genres,cast,crew,keywords,movieId,imdbId
0,862,Toy Story,1995-10-30,5415,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",1,114709


In [16]:
df = df.drop(columns='movieId')

In [17]:
df.shape

(9219, 10)



---

# Work with categorical data

---



---
## Handling 'genres' column
---

In [18]:
def handling_genres_column(data):
  data['genres'] = data['genres'].fillna('[]') \
                             .apply(ast.literal_eval) \
                             .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

  data['genres'] = [', '.join(i) for i in data['genres']]
  data['genres'] = pd.DataFrame(data=map(str.lower, data['genres'])) 
  data['genres'] = [i.replace(",", "") for i in data['genres']]

  return data

In [21]:
df = handling_genres_column(df)

In [22]:
df.genres.head()

0     animation comedy family
1    adventure fantasy family
2              romance comedy
3        comedy drama romance
4                      comedy
Name: genres, dtype: object



---

## Handling 'cast' column

---



In [23]:
def handling_cast_column(data):
  data['cast'] = data['cast'].fillna('[]') \
                             .apply(ast.literal_eval) \
                             .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

  data['cast'] = [', '.join(i) for i in data['cast']]
  data['cast'] = pd.DataFrame(data=map(str.lower, data['cast'])) 
  data['cast'] = [i.replace(" ", "") for i in data['cast']]
  data['cast'] = [i.replace(",", " ") for i in data['cast']]

  return data

In [24]:
df = handling_cast_column(df)

In [25]:
df.cast.head()

0    tomhanks timallen donrickles jimvarney wallace...
1    robinwilliams jonathanhyde kirstendunst bradle...
2    waltermatthau jacklemmon ann-margret sophialor...
3    whitneyhouston angelabassett lorettadevine lel...
4    stevemartin dianekeaton martinshort kimberlywi...
Name: cast, dtype: object



---

## Handling 'crew' column

---



In [26]:
df['crew'] = df['crew'].apply(ast.literal_eval)

In [27]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [28]:
df['director'] = df['crew'].apply(get_director)
df['director'] = df['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
df.director.head()

0      johnlasseter
1       joejohnston
2      howarddeutch
3    forestwhitaker
4      charlesshyer
Name: director, dtype: object

In [29]:
df = df.drop(columns='crew')



---

## Handling 'keywords' column

---



In [33]:
def handling_keywords_column(data):
  data['keywords'] = data['keywords'].fillna('[]') \
                              .apply(ast.literal_eval) \
                              .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
  return data

In [34]:
df = handling_keywords_column(df)

In [35]:
df.keywords.head()

0    [jealousy, toy, boy, friendship, friends, riva...
1    [board game, disappearance, based on children'...
2    [fishing, best friend, duringcreditsstinger, o...
3    [based on novel, interracial relationship, sin...
4    [baby, midlife crisis, confidence, aging, daug...
Name: keywords, dtype: object



---

### Stemming of 'keywords' column

---



In [36]:
stemmer = SnowballStemmer(language='english')

In [37]:
df['keywords'] = df['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
df['keywords'] = [', '.join(i) for i in df['keywords']] 
df['keywords'] = [i.replace(" ", "") for i in df['keywords']]
df['keywords'] = [i.replace(",", " ") for i in df['keywords']]

In [38]:
df.keywords.head()

0    jealousi toy boy friendship friend rivalri boy...
1    boardgam disappear basedonchildren'sbook newho...
2            fish bestfriend duringcreditssting oldmen
3    basedonnovel interracialrelationship singlemot...
4    babi midlifecrisi confid age daughter motherda...
Name: keywords, dtype: object



---
# CountVectorizer
---





In [40]:
def concat_fields(data):
    concat = data['genres'] + ' ' + data['cast'] + ' ' + data['keywords']  + ' ' + data['director'] 
    result = ''.join([str(i).lower() for i in concat])
    return result

In [41]:
df['information'] = df.apply(lambda x: concat_fields(x), axis=1)

In [42]:
df.information.head()

0    animation comedy family tomhanks timallen donr...
1    adventure fantasy family robinwilliams jonatha...
2    romance comedy waltermatthau jacklemmon ann-ma...
3    comedy drama romance whitneyhouston angelabass...
4    comedy stevemartin dianekeaton martinshort kim...
Name: information, dtype: object

In [43]:
df.information[0]

'animation comedy family tomhanks timallen donrickles jimvarney wallaceshawn johnratzenberger anniepotts johnmorris erikvondetten lauriemetcalf r.leeermey sarahfreeman pennjillette jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif johnlasseter'



---
# Arrangement of columns in the dataset
---





In [51]:
df = df[['id', 'imdbId', 'title', 'release_date', 'vote_count', 'vote_average', 
         'genres', 'cast', 'keywords', 'director', 'information']]

In [52]:
df.head()

,id,imdbId,title,release_date,vote_count,vote_average,genres,cast,keywords,director,information
0,862,114709,Toy Story,1995-10-30,5415,7.7,animation comedy family,tomhanks timallen donrickles jimvarney wallace...,jealousi toy boy friendship friend rivalri boy...,johnlasseter,animation comedy family tomhanks timallen donr...
1,8844,113497,Jumanji,1995-12-15,2413,6.9,adventure fantasy family,robinwilliams jonathanhyde kirstendunst bradle...,boardgam disappear basedonchildren'sbook newho...,joejohnston,adventure fantasy family robinwilliams jonatha...
2,15602,113228,Grumpier Old Men,1995-12-22,92,6.5,romance comedy,waltermatthau jacklemmon ann-margret sophialor...,fish bestfriend duringcreditssting oldmen,howarddeutch,romance comedy waltermatthau jacklemmon ann-ma...
3,31357,114885,Waiting to Exhale,1995-12-22,34,6.1,comedy drama romance,whitneyhouston angelabassett lorettadevine lel...,basedonnovel interracialrelationship singlemot...,forestwhitaker,comedy drama romance whitneyhouston angelabass...
4,11862,113041,Father of the Bride Part II,1995-02-10,173,5.7,comedy,stevemartin dianekeaton martinshort kimberlywi...,babi midlifecrisi confid age daughter motherda...,charlesshyer,comedy stevemartin dianekeaton martinshort kim...


---
# Recommendation
---

In [45]:
vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=2)
count_matrix = vectorizer.fit_transform(df['information'])
count_matrix.shape

(9219, 43875)

In [46]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim.shape

(9219, 9219)

In [47]:
titles = df.title
indices = pd.Series(df.index, index=df['title'])

In [48]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [49]:
get_recommendations('The Dark Knight').head(10)

6218                      Batman Begins
8031              The Dark Knight Rises
9121                       İtirazım Var
1134                     Batman Returns
7659         Batman: Under the Red Hood
8927            Kidnapping Mr. Heineken
7362    Gangster's Paradise: Jerusalema
5943                           Thursday
4406                            Brother
5834                 Gone in 60 Seconds
Name: title, dtype: object